# <span style="color:turquoise">**Detección de células redondas utilizando Yolo**</span>

## **Hardware**

**PC** Lenovo Idealpad Gaming3

**Procesador** AMD Ryzen 5 5600H with Radeon Graphics 3.3 GHz

	Velocidad de base:	3,30 GHz
	Sockets:	1
	Núcleos:	6
	Procesadores lógicos:	12
	Virtualización:	Habilitado
	Caché L1:	384 kB
	Caché L2:	3,0 MB
	Caché L3:	16,0 MB

**Gráficos dedicados** NVIDIA GeForce RTX 3050 Laptop 

	Versión del controlador:	31.0.15.5176
	Fecha del controlador:	01/03/2024
	Versión de DirectX:	12 (FL 12.1)
	Memoria de GPU dedicada	4,0 GB
	Memoria de GPU compartida	7,7 GB
	Memoria de GPU	11,7 GB
	
**RAM** 16 GB SODIMM DDR4 

	Velocidad:	3200 MHz
	Ranuras usadas:	2 de 2
	Factor de forma:	SODIMM

**Memoria estado sólido** SSD 512 GB 2210 NVMe PCIe Gen3 x4  Micron MTFDHBA512QFD

**Google Colab** 
	12.7 GB RAM, 15 GB RAM GPU Tesla T4, SSD 78.2 GB

In [1]:
!nvidia-smi

Fri Jul 18 00:22:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              6W /   60W |     555MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## **Introducción**

En este TFM se pretende realizar un estudio del arte y entrenamiento de diferentes modelos (YOLO, Fast-RCNN, propio) para la detección de células y artefactos en imágenes médicas. Para esto, se aplican técnicas de visión por computador y aprendizaje profundo. Además, se realizará un exahustiva revisión de la bibliografía existente, mejora del dataset y análisis estadistico fundamentado en las diferentes métricas.

## **Dependencias**

In [1]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

import torch
from ultralytics import YOLO

## **Entrenamineto de los modelos YOLOv11 y YOLOv12**

In [2]:
print("PyTorch ha sido importado con éxito.")
print(f"Versión de PyTorch: {torch.__version__}")
print(f"¿CUDA está disponible? {torch.cuda.is_available()}")
print(f"Versión de CUDA: {torch.version.cuda}")
print(f"Dispositivo CUDA: {torch.cuda.get_device_name(0)}")

PyTorch ha sido importado con éxito.
Versión de PyTorch: 2.6.0+cu126
¿CUDA está disponible? True
Versión de CUDA: 12.6
Dispositivo CUDA: NVIDIA GeForce RTX 3050 Laptop GPU


In [7]:
# %pip install ultralytics

------------------------------------------------------------------------------------

In [ ]:
# !pip install optuna

In [3]:
import optuna

def objective(trial):
    params = {
        # 🎯 OPTIMIZACIÓN
        'lr0': trial.suggest_float('lr0', 0.0001, 0.01, log=True),
        'lrf': trial.suggest_float('lrf', 0.001, 0.01, log=True),
        'momentum': trial.suggest_float('momentum', 0.8, 0.95),
        'weight_decay': trial.suggest_float('weight_decay', 0.0001, 0.001, log=True), # YOLO solo tiene regulación L2
        'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW']),

        'warmup_epochs': 5,
        'warmup_momentum': 0.75,

        # 📐 AUGMENTACIÓN GEOMÉTRICA
        'degrees': 45,
        'translate': 0.1,
        'flipud': 0.5,
        'fliplr': 0.5,
        'mosaic': 0,
        'close_mosaic': 0,
        # Interesantes: 'label_smoothing', 'mixup', 'cutmix'

        # ⚖️ PÉRDIDAS
        # 'box': trial.suggest_float('box', 0.02, 0.2),
        # 'cls': trial.suggest_float('cls', 0.2, 4.0),
        # 'dfl': trial.suggest_float('dfl', 0.5, 3.0),
    }

    trial_name = f"optuna_trial_{trial.number}"

    try:
        model = YOLO("yolo_models/yolo12m.pt")
        results = model.train(
            data="cells.yaml",
            epochs=25,
            imgsz=704,
            batch=4,
            name=trial_name,
            save=False,
            verbose=False,
            **params
        )
        metrics = results.results_dict
        return metrics.get('metrics/mAP50-95(B)', 0.0)
    except Exception as e:
        print(f"Error en trial {trial.number}: {str(e)}")
        return 0.0

c:\Users\mini7\miniconda3\envs\TFM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

study_name = "pruebav12m"
n_trials = 8
storage_name = f"sqlite:///optuna/optuna_studies/{study_name}.db"

study = optuna.create_study(
    direction="maximize",  # Maximizar mAP
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42)  # Usar TPE sampler para mejor eficiencia
)

# Ejecutar optimización
study.optimize(objective, n_trials=n_trials, timeout=None)

# Mostrar resultados
print("Número de trials completados:", len(study.trials))
print("Mejor trial:")
trial = study.best_trial
print(f"  Valor: {trial.value}")
print("  Parámetros:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Guardar mejores parámetros
best_params_path = f"optuna/optuna_studies/{study_name}_best_params.json"
with open(best_params_path, 'w') as f:
    json.dump(trial.params, f, indent=2)

print(f"Mejores parámetros guardados en: {best_params_path}")

In [ ]:
with open("optuna/optuna_studies/prueba5V12S_best_params.json", "r") as f:
    best_params = json.load(f)

print("Entrenando modelo final con mejores parámetros...")
print("Parámetros:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

model = YOLO("yolo_models/yolo12m.pt")

model_name = "final_model_optunav12m"
results = model.train(
    data="cells.yaml",
    epochs=60,         # Ajusta según tus recursos y necesidades
    imgsz=704,         # Tamaño de imagen para el entrenamiento final
    batch=3,           # Ajusta según tu GPU/CPU
    name=model_name,
    save=True,         # Guarda los checkpoints y resultados
    **best_params      # Aplica los hiperparámetros óptimos
)


In [ ]:
# 1. Historia de optimización
plt.figure(figsize=(12, 8))

# Gráfico 1: Valor objetivo por trial
plt.subplot(2, 2, 1)
trials = [trial.value for trial in study.trials if trial.value is not None]
plt.plot(trials, marker='o', markersize=3)
plt.title('Valor Objetivo por Trial')
plt.xlabel('Trial')
plt.ylabel('mAP@0.5:0.95')
plt.grid(True, alpha=0.3)

# Gráfico 2: Mejores valores acumulados
plt.subplot(2, 2, 2)
best_values = []
current_best = float('-inf')
for trial in study.trials:
    if trial.value is not None and trial.value > current_best:
        current_best = trial.value
    best_values.append(current_best if current_best != float('-inf') else 0)

plt.plot(best_values, marker='o', markersize=3, color='red')
plt.title('Mejor Valor Acumulado')
plt.xlabel('Trial')
plt.ylabel('Mejor mAP@0.5:0.95')
plt.grid(True, alpha=0.3)

# Gráfico 3: Distribución de valores
plt.subplot(2, 2, 3)
plt.hist(trials, bins=20, alpha=0.7, edgecolor='black')
plt.title('Distribución de Valores Objetivo')
plt.xlabel('mAP@0.5:0.95')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

# Gráfico 4: Top 10 parámetros más importantes
plt.subplot(2, 2, 4)
try:
    importance = optuna.importance.get_param_importances(study)
    top_params = dict(sorted(importance.items(), key=lambda x: x[1], reverse=True)[:10])
    
    plt.barh(list(top_params.keys()), list(top_params.values()))
    plt.title('Top 10 Parámetros Más Importantes')
    plt.xlabel('Importancia')
    plt.tight_layout()
except:
    plt.text(0.5, 0.5, 'No disponible\n(necesita más trials)', 
            ha='center', va='center', transform=plt.gca().transAxes)

plt.tight_layout()
plt.savefig('optuna/optuna_plots/optimization_resultsV12m.png', dpi=300, bbox_inches='tight')
plt.show()

<Figure size 1200x800 with 4 Axes>

----------------------------------------------------------------------------------------

In [ ]:
# # YOLO 11
# model_YOLOv11n = YOLO("yolo_models/yolo11n.pt")
# model_YOLOv11s = YOLO("yolo_models/yolo11s.pt")
# model_YOLOv11m = YOLO("yolo_models/yolo11m.pt")
# model_YOLOv11l = YOLO("yolo_models/yolo11l.pt")
# model_YOLOv11x = YOLO("yolo_models/yolo11x.pt")
# model_YOLOv11ny = YOLO("yolo_models/yolo11n.yaml")

# # YOLO 12
# model_YOLOv12n = YOLO("yolo_models/yolo12n.pt")
# model_YOLOv12s = YOLO("yolo_models/yolo12s.pt")
# model_YOLOv12m = YOLO("yolo_models/yolo12m.pt")
# model_YOLOv12l = YOLO("yolo_models/yolo12l.pt")
# model_YOLOv12x = YOLO("yolo_models/yolo12x.pt")

In [4]:
exp_paths = {
    'yolov12n': 'runs/detect/yolov12n/results.csv',
    'yolov11n': 'runs/detect/yolov11n/results.csv',
    'yolov11l': 'runs/detect/yolov11l/results.csv',
    'yolov11s': 'runs/detect/yolov11s/results.csv',
    'yolov11ny': 'runs/detect/yolov11ny/results.csv',
    'yolov11ny_new': 'runs/detect/yolov11ny/results.csv',
    'yolooptunaV11S': 'runs/detect/final_model_optunav11s/results.csv',
    'yolooptunaV12S': 'runs/detect/final_model_optunav12s/results.csv',
    # 'yolooptunaV12m': 'runs/detect/final_model_optunav12m/results.csv',
}

plots_dir = 'yolo_plots'
os.makedirs(plots_dir, exist_ok=True)

# Cargar los datos
dfs = {}
for model_name, path in exp_paths.items():
    try:
        dfs[model_name] = pd.read_csv(path)
    except Exception as e:
        print(f"Error al cargar {path}: {e}")

def create_and_save_individual_plot(plot_type, y_column, title, ylabel, is_dual=False, y_limit=None):
    plt.figure(figsize=(10, 6))
    
    if is_dual: 
        for model_name, df in dfs.items():
            plt.plot(df['epoch'], df[y_column[0]], label=f"{model_name} - Train")
            plt.plot(df['epoch'], df[y_column[1]], '--', label=f"{model_name} - Val")
    else:  
        for model_name, df in dfs.items():
            plt.plot(df['epoch'], df[y_column], label=model_name)
    
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    
    if y_limit:
        plt.ylim(*y_limit)
    
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.tight_layout()
    
    filename = f"{plots_dir}/{plot_type}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"La figura individual '{plot_type}' se ha guardado en: {filename}")
    plt.close()

create_and_save_individual_plot('map50-95', 'metrics/mAP50-95(B)', 'Training mAP@0.5:0.95', 'mAP@0.5:0.95')
create_and_save_individual_plot('map50', 'metrics/mAP50(B)', 'Training mAP@0.5', 'mAP@0.5')
create_and_save_individual_plot('precision', 'metrics/precision(B)', 'Val Precision', 'Precision')
create_and_save_individual_plot('recall', 'metrics/recall(B)', 'Val Recall', 'Recall')
create_and_save_individual_plot('box_loss', ['train/box_loss', 'val/box_loss'], 
                               'Training & Validation Box Loss', 'Box Loss', True, (0, 4))
create_and_save_individual_plot('cls_loss', ['train/cls_loss', 'val/cls_loss'], 
                               'Training & Validation Classification Loss', 'Cls Loss', True, (0, 4))

fig, axes = plt.subplots(3, 2, figsize=(14, 18))
axes = axes.ravel() 

##############################################################################
# 1) mAP@0.5:0.95
##############################################################################
for model_name, df in dfs.items():
    # Ajusta el nombre de la columna a la que aparezca en tu CSV
    # En YOLOv5 suele ser 'metrics/mAP_0.5:0.95'
    axes[0].plot(df['epoch'], df['metrics/mAP50-95(B)'], label=model_name)

axes[0].set_title('Training mAP@0.5:0.95')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('mAP@0.5:0.95')
axes[0].legend()

##############################################################################
# 2) mAP@0.5
##############################################################################
for model_name, df in dfs.items():
    # En YOLOv5 suele ser 'metrics/mAP_0.5'
    axes[1].plot(df['epoch'], df['metrics/mAP50(B)'], label=model_name)

axes[1].set_title('Training mAP@0.5')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('mAP@0.5')
axes[1].legend()

##############################################################################
# 3) Precision
##############################################################################
for model_name, df in dfs.items():
    # Suele ser 'metrics/precision'
    axes[2].plot(df['epoch'], df['metrics/precision(B)'], label=model_name)

axes[2].set_title('Val Precision')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Precision')
axes[2].legend()

##############################################################################
# 4) Recall
##############################################################################
for model_name, df in dfs.items():
    # Suele ser 'metrics/recall'
    axes[3].plot(df['epoch'], df['metrics/recall(B)'], label=model_name)

axes[3].set_title('Val Recall')
axes[3].set_xlabel('Epoch')
axes[3].set_ylabel('Recall')
axes[3].legend()

##############################################################################
# 5) Box Loss (Train vs Val)
##############################################################################
for model_name, df in dfs.items():
    # Columnas típicas: 'loss/box' (entrenamiento) y 'val/box_loss' (validación)
    axes[4].plot(df['epoch'], df['train/box_loss'], label=f"{model_name} - Train")
    axes[4].plot(df['epoch'], df['val/box_loss'], '--', label=f"{model_name} - Val")

axes[4].set_title('Training & Validation Box Loss')
axes[4].set_xlabel('Epoch')
axes[4].set_ylabel('Box Loss')
axes[4].set_ylim(0, 4)
axes[4].legend()

##############################################################################
# 6) Classification Loss (Train vs Val)
##############################################################################
for model_name, df in dfs.items():
    # Columnas típicas: 'loss/cls' (entrenamiento) y 'val/cls_loss' (validación)
    axes[5].plot(df['epoch'], df['train/cls_loss'], label=f"{model_name} - Train")
    axes[5].plot(df['epoch'], df['val/cls_loss'], '--', label=f"{model_name} - Val")

axes[5].set_title('Training & Validation Classification Loss')
axes[5].set_xlabel('Epoch')
axes[5].set_ylabel('Cls Loss')
axes[5].set_ylim(0, 4)
axes[5].legend()

##############################################################################
# Ajustamos el layout y guardamos la figura (en lugar de mostrarla)
##############################################################################
plt.tight_layout()

output_path = 'plots/yolov_training_metrics.png'

os.makedirs('plots', exist_ok=True)  
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"La figura se ha guardado en: {output_path}")
plt.close(fig)

La figura individual 'map50-95' se ha guardado en: yolo_plots/map50-95.png
La figura individual 'map50' se ha guardado en: yolo_plots/map50.png
La figura individual 'precision' se ha guardado en: yolo_plots/precision.png
La figura individual 'recall' se ha guardado en: yolo_plots/recall.png
La figura individual 'box_loss' se ha guardado en: yolo_plots/box_loss.png
La figura individual 'cls_loss' se ha guardado en: yolo_plots/cls_loss.png
La figura se ha guardado en: plots/yolov_training_metrics.png


## **Evaluaciones de empresa**

In [17]:
def process_images(directorio_entrada, directorio_referencia, directorio_salida, modelo_path):
    """
    Process all images in a directory with YOLO and compare with reference images.
    
    Args:
        directorio_entrada: Path to directory containing input images
        directorio_referencia: Path to directory containing reference images
        directorio_salida: Path to save output comparison images
        modelo_path: Path to YOLO model weights
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
        print(f"Directorio creado: {directorio_salida}")
    else:
        print(f"Usando directorio de salida: {directorio_salida}")
    
    modelo_personalizado = YOLO(modelo_path)
    
    archivos_entrada = [f for f in os.listdir(directorio_entrada) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not archivos_entrada:
        print(f"No se encontraron imágenes en {directorio_entrada}")
        return
    
    print(f"Procesando {len(archivos_entrada)} imágenes...")
    
    for nombre_archivo in archivos_entrada:
        print(f"Procesando: {nombre_archivo}")
        ruta_imagen = os.path.join(directorio_entrada, nombre_archivo)
        
        nombre_base = os.path.splitext(nombre_archivo)[0]
        nombre_referencia = f"{nombre_base}_.jpg"
        ruta_imagen_referencia = os.path.join(directorio_referencia, nombre_referencia)
        
        if not os.path.exists(ruta_imagen_referencia):
            print(f"Advertencia: No se encontró la imagen de referencia {nombre_referencia}")
            ruta_imagen_referencia = None
        
        resultados = modelo_personalizado(ruta_imagen)
        
        for r in resultados:
            num_detecciones = len(r.boxes)

            im_array = r.plot(labels=False, conf=True)

            font = cv2.FONT_HERSHEY_SIMPLEX
            texto_contador = f"{num_detecciones}"

            # Calculamos el tamaño del texto para posicionarlo correctamente
            textSize = cv2.getTextSize(texto_contador, font, 0.8, 2)[0]
            
            # Posicionamos en la esquina superior derecha con un margen de 20px
            text_x = im_array.shape[1] - textSize[0] - 20
            text_y = 30  # 30px desde la parte superior
            
            # Dibujamos el texto
            cv2.putText(im_array, texto_contador, (text_x, text_y), font, 0.8, (255, 255, 255), 2)
            
            if ruta_imagen_referencia:
                img_referencia = cv2.imread(ruta_imagen_referencia)
                
                h1, w1 = im_array.shape[:2]
                h2, w2 = img_referencia.shape[:2]
                
                if h1 != h2:
                    if h1 > h2:
                        new_w2 = int(w2 * (h1 / h2))
                        img_referencia = cv2.resize(img_referencia, (new_w2, h1))
                    else:
                        new_w1 = int(w1 * (h2 / h1))
                        im_array = cv2.resize(im_array, (new_w1, h2))
                
                img_combinada = cv2.hconcat([im_array, img_referencia])
                
                cv2.line(img_combinada, (im_array.shape[1], 0), (im_array.shape[1], img_combinada.shape[0]), (90, 70, 255), 2)
                
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img_combinada, "YOLO", (10, 30), font, 1, (0, 255, 0), 2)
                cv2.putText(img_combinada, "IR", (im_array.shape[1] + 10, 30), font, 1, (0, 255, 0), 2)
                
                ruta_salida = os.path.join(directorio_salida, f"comparacion_{nombre_archivo}")
                cv2.imwrite(ruta_salida, img_combinada)
            else:
                ruta_salida = os.path.join(directorio_salida, f"yolo_{nombre_archivo}")
                cv2.imwrite(ruta_salida, im_array)
            
    print(f"Procesamiento completado. Resultados guardados en {directorio_salida}")

Añadir la evaluación sobre el conjunto de test

In [18]:
modelo_path = '.\\runs\\detect\\final_model_optunav11s2\\weights\\best.pt'

# DEFAULT dataset
process_images(
    "..\\03.Datasets\\Evaluacion_Empresa\\DEFAULT\\INPUT DEFAULT",
    "..\\03.Datasets\\Evaluacion_Empresa\\DEFAULT\\OUTPUT_edited",
    "..\\03.Datasets\\Evaluacion_Empresa\\DEFAULT\\YOLO_NEW",
    modelo_path
)

# TEST 2
process_images(
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 2\\INPUT TEST_2",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 2\\OUTPUT_TEST 2_edited",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 2\\YOLO_NEW",
    modelo_path
)

# TEST 3
process_images(
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 3\\INPUT_TEST 3",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 3\\OUTPUT_TEST 3_edited",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 3\\YOLO_NEW",
    modelo_path
)

# TEST 4
process_images(
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 4\\INPUT_TEST 4",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 4\\OUTPUT_TEST 4_edited",
    "..\\03.Datasets\\Evaluacion_Empresa\\TEST 4\\YOLO_NEW",
    modelo_path
)

Directorio creado: ..\03.Datasets\Evaluacion_Empresa\DEFAULT\YOLO_NEW
Procesando 94 imágenes...
Procesando: 110.jpg
Advertencia: No se encontró la imagen de referencia 110_.jpg

image 1/1 c:\Users\mini7\Desktop\Master\Materias\TFM\Laura\2025_MURIA_Aitor Garca Blanco\04.Codigo\..\03.Datasets\Evaluacion_Empresa\DEFAULT\INPUT DEFAULT\110.jpg: 576x704 2 cells, 55.2ms
Speed: 7.6ms preprocess, 55.2ms inference, 4.0ms postprocess per image at shape (1, 3, 576, 704)
Procesando: 116.jpg

image 1/1 c:\Users\mini7\Desktop\Master\Materias\TFM\Laura\2025_MURIA_Aitor Garca Blanco\04.Codigo\..\03.Datasets\Evaluacion_Empresa\DEFAULT\INPUT DEFAULT\116.jpg: 576x704 35 cells, 26.2ms
Speed: 3.8ms preprocess, 26.2ms inference, 4.0ms postprocess per image at shape (1, 3, 576, 704)
Procesando: 124.jpg
Advertencia: No se encontró la imagen de referencia 124_.jpg

image 1/1 c:\Users\mini7\Desktop\Master\Materias\TFM\Laura\2025_MURIA_Aitor Garca Blanco\04.Codigo\..\03.Datasets\Evaluacion_Empresa\DEFAULT\INPUT D

Añadir metricas de IOU, accuracy, etc

Usar optuna en lugar de grid search u random search

añadir kfold y una función general que importe con un import para generar una carpeta con los resultados de la evaluación de cada imagen del test/defalut/eval_empresa 

Probar a entrenar Yolo con capas de atención y mucho más personalizados algunos modelos.

pODEMOS AÑADIR UN ENTORNO WEB DONDE ADJUNTAMOS LA IMAGEN Y SE MUESTRA EL RESULTADO DE LAS DIFERENTES PREDICCIONES.

 hacer RFHF con yolo?

## **Conclusión**

## **Comentarios reunión semanal**

Para el power point usar el mes gratis de copilot, así lo redacto mejor y más rápido la presentación

"no es posible distinguir entre leucocitos y células germinales inmaduras con un alto grado de certeza (2)" https://pmc.ncbi.nlm.nih.gov/articles/PMC8915675/

Quiero probar este modelo: https://github.com/Peterande/D-FINE?tab=readme-ov-file